<a href="https://colab.research.google.com/github/Palakjhamnani/multilingual-Recommendation-vlg-project/blob/main/Multilingual_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install aicrowd-cli pyarrow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.8 MB/s eta 0:00:00
  Created wheel for pyzmq: filename=pyzmq-22.1.0-cp310-cp310-linux_x86_64.whl size=1564516 sha256=1abe52f50a4afd5856a51f528f0

In [ ]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/m3KY-gFTnBQ5Kk8gOpY9DtZ2LM7isKhpHs9zIZAj0mY
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/m3KY-gFTnBQ5Kk8gOpY9DtZ2LM7isKhpHs9zIZAj0mY'
API Key valid
Gitlab access token valid
Saved details successfully!


In [ ]:
!aicrowd dataset download --challenge task-1-next-product-recommendation

sessions_test_task1_phase1.csv: 100% 19.4M/19.4M [00:02<00:00, 9.50MB/s]
sessions_test_task2_phase1.csv: 100% 1.92M/1.92M [00:00<00:00, 5.95MB/s]
sessions_test_task3_phase1.csv: 100% 2.67M/2.67M [00:00<00:00, 7.55MB/s]
sessions_test_task1.csv: 100% 19.3M/19.3M [00:01<00:00, 11.2MB/s]
sessions_test_task2.csv: 100% 1.91M/1.91M [00:00<00:00, 5.87MB/s]
sessions_test_task3.csv: 100% 2.67M/2.67M [00:00<00:00, 7.26MB/s]
products_train.csv: 100% 589M/589M [01:15<00:00, 7.83MB/s]
sessions_train.csv: 100% 259M/259M [00:40<00:00, 6.36MB/s]


In [ ]:
import os
import numpy as np
import pandas as pd
from functools import lru_cache

In [ ]:
train_data_dir = '.'
test_data_dir = '.'
task = 'task1'
PREDS_PER_SESSION = 100

In [ ]:
@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(os.path.join(train_data_dir, 'products_train.csv'))

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(os.path.join(train_data_dir, 'sessions_train.csv'))

@lru_cache(maxsize=3)
def read_test_data(task):
    return pd.read_csv(os.path.join(test_data_dir, f'sessions_test_{task}.csv'))

In [ ]:
def read_locale_data(locale, task):
    products = read_product_data().query(f'locale == "{locale}"')
    sess_train = read_train_data().query(f'locale == "{locale}"')
    sess_test = read_test_data(task).query(f'locale == "{locale}"')
    return products, sess_train, sess_test

def show_locale_info(locale, task):
    products, sess_train, sess_test = read_locale_data(locale, task)

    train_l = sess_train['prev_items'].apply(lambda sess: len(sess))
    test_l = sess_test['prev_items'].apply(lambda sess: len(sess))

    print(f"Locale: {locale} \n"
          f"Number of products: {products['id'].nunique()} \n"
          f"Number of train sessions: {len(sess_train)} \n"
          f"Train session lengths - "
          f"Mean: {train_l.mean():.2f} | Median {train_l.median():.2f} | "
          f"Min: {train_l.min():.2f} | Max {train_l.max():.2f} \n"
          f"Number of test sessions: {len(sess_test)}"
        )
    if len(sess_test) > 0:
        print(
             f"Test session lengths - "
            f"Mean: {test_l.mean():.2f} | Median {test_l.median():.2f} | "
            f"Min: {test_l.min():.2f} | Max {test_l.max():.2f} \n"
        )
    print("======================================================================== \n")

In [ ]:
products = read_product_data()
locale_names = products['locale'].unique()
for locale in locale_names:
    show_locale_info(locale, task)

Locale: DE 
Number of products: 518327 
Number of train sessions: 1111416 
Train session lengths - Mean: 57.89 | Median 40.00 | Min: 27.00 | Max 2060.00 
Number of test sessions: 104568
Test session lengths - Mean: 56.91 | Median 40.00 | Min: 27.00 | Max 1043.00 


Locale: JP 
Number of products: 395009 
Number of train sessions: 979119 
Train session lengths - Mean: 59.61 | Median 40.00 | Min: 27.00 | Max 6257.00 
Number of test sessions: 96467
Test session lengths - Mean: 59.84 | Median 40.00 | Min: 27.00 | Max 1466.00 


Locale: UK 
Number of products: 500180 
Number of train sessions: 1182181 
Train session lengths - Mean: 54.85 | Median 40.00 | Min: 27.00 | Max 2654.00 
Number of test sessions: 115937
Test session lengths - Mean: 53.25 | Median 40.00 | Min: 27.00 | Max 753.00 


Locale: ES 
Number of products: 42503 
Number of train sessions: 89047 
Train session lengths - Mean: 48.82 | Median 40.00 | Min: 27.00 | Max 792.00 
Number of test sessions: 0

Locale: FR 
Number of produ

In [ ]:
products.sample(5)

,id,locale,title,price,brand,color,size,model,material,author,desc
543136,B07V9QWJZP,JP,パナソニック カエバ メンズシェーバーヨウ ソトバ,3800.00,パナソニック(Panasonic),NaN,NaN,ES9179,NaN,NaN,型番:ES9179
1499443,B096BLZB16,FR,Lizzy Costume d'Halloween pour enfants avec ai...,23.56,Lizzy,Dark Angel Wings Halo Tutu Noir,Taille unique,NaN,Polyester,NaN,"Ailes d'ange en plumes véritables, serre-tête,..."
173281,B0948F9TK5,DE,"Tastatur Hülle für iPad Pro 12,9 Zoll 1. 2. Ge...",48.99,HaoHZ,Black,NaN,NaN,Acrylnitril-Butadien-Styrol,NaN,★ 100% Zufriedenheitsgarantie: Unser Team biet...
1156255,B07HZ9VX3V,UK,"Rexel Momentum X312 Cross Cut Paper Shredder, ...",80.99,Rexel,Black,Momentum,2104572,NaN,NaN,"2-year warranty against defects, simple intuit..."
672158,B00MTIDRCO,JP,パールキンゾク クラディア ホーローガラスフタヨセシャブナベ20cm レッド,2378.00,パール金属(PEARL METAL),レッド,20cm,HB-936,本体・取っ手/ほうろう用鋼板(ほうろう加工)、本体枠部/ステンレス鋼、ガラス蓋/強化ガラス(...,NaN,原産国:中国


In [ ]:
train_sessions = read_train_data()
train_sessions.sample(5)

,prev_items,next_item,locale
2443862,['B08T19F29G' 'B08T1XQM1X' 'B08T1F4HRZ' 'B08T1...,B08SR7JJX4,UK
926899,['B00DAX14FE' 'B00C3XJL7A' 'B00C3XJL7A' 'B00DA...,B07XBZ4W9P,DE
3399187,['B0B139RW78' 'B09W68119C' 'B08YK36HR7' 'B08DR...,B09ZLQ6GZW,FR
2960503,['B09VZMCCZT' 'B09VZKPX1X'],B09K3LQKYW,UK
2184911,['B09NLB7SZ6' 'B08XY4WGCJ'],B09X3JSZLK,UK


In [ ]:
test_sessions = read_test_data(task)
test_sessions.sample(5)


,prev_items,locale
244577,['B07C21PCCW' 'B0932XV1FW'],UK
152859,['B09Q853SSX' 'B0B37GZ479' 'B07ZTCS96Q' 'B09BY...,JP
111852,['B00HZV9XKU' 'B00HZV9YE0' 'B08T8BYH28'],JP
230935,['B0792YHNKZ' 'B00E9T48NI' 'B00E9T48NI'],UK
176128,['B0B5DRJ2JY' 'B09PBPTJ8M' 'B09PDN3ZSR'],JP


In [ ]:
def random_predicitons(locale, sess_test_locale):
    random_state = np.random.RandomState(42)
    products = read_product_data().query(f'locale == "{locale}"')
    predictions = []
    for _ in range(len(sess_test_locale)):
        predictions.append(
            list(products['id'].sample(PREDS_PER_SESSION, replace=True, random_state=random_state))
        )
    sess_test_locale['next_item_prediction'] = predictions
    sess_test_locale.drop('prev_items', inplace=True, axis=1)
    return sess_test_locale

In [ ]:
test_sessions = read_test_data(task)
predictions = []
test_locale_names = test_sessions['locale'].unique()
for locale in test_locale_names:
    sess_test_locale = test_sessions.query(f'locale == "{locale}"').copy()
    predictions.append(
        random_predicitons(locale, sess_test_locale)
    )
predictions = pd.concat(predictions).reset_index(drop=True)
predictions.sample(5)

,locale,next_item_prediction
146833,JP,"[B07JNFLDGF, B08KDRFM7H, B09CGP59YN, B09C284K1..."
249988,UK,"[B08RWK3N1J, B0BGKZ314D, B0B49PRG1C, B0B2QT2C2..."
252947,UK,"[B087JYPC6D, B09F9L7DJH, B0B3WSNJ7S, B08LL5DTS..."
303787,UK,"[B09CH4TR8T, B017V1CLSO, B07VMHRYJN, B0BDJGV88..."
55994,DE,"[B09QQH3BDR, B07HY91ZN2, B07978YC3J, B07ZK2TT7..."


In [ ]:
def check_predictions(predictions, check_products=False):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_sessions['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_sessions.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"

        if check_products:
            # This check is not done on the evaluator
            # but you can run it to verify there is no mixing of products between locales
            # Since the ground truth next item will always belong to the same locale
            # Warning - This can be slow to run
            products = read_product_data().query(f'locale == "{locale}"')
            predicted_products = np.unique( np.array(list(preds_locale["next_item_prediction"].values)) )
            assert np.all( np.isin(predicted_products, products['id']) ), f"Invalid products in {locale} predictions"

In [ ]:
check_predictions(predictions)

In [ ]:
predictions.to_parquet(f'submission_{task}.parquet', engine='pyarrow')


In [ ]:
!aicrowd submission create -c task-1-next-product-recommendation -f "submission_task1.parquet"





Streaming output truncated to the last 5000 lines.
submission_task1.parquet ━━━━━━━━━━╺━ 87.3% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.3% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.3% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2   • 2.0 MB/s • 0:00:20
submission_task1.parquet ━━━━━━━━━━╺━ 87.4% • 270.1/309.2 

# task 2


In [ ]:
!aicrowd dataset download --challenge task-2-next-product-recommendation-for-underrepresented-languages

sessions_test_task1_phase1.csv: 100% 19.4M/19.4M [00:02<00:00, 7.99MB/s]
sessions_test_task2_phase1.csv: 100% 1.92M/1.92M [00:00<00:00, 5.65MB/s]
sessions_test_task3_phase1.csv: 100% 2.67M/2.67M [00:00<00:00, 7.57MB/s]
sessions_test_task1.csv: 100% 19.3M/19.3M [00:00<00:00, 28.8MB/s]
sessions_test_task2.csv: 100% 1.91M/1.91M [00:00<00:00, 5.53MB/s]
sessions_test_task3.csv: 100% 2.67M/2.67M [00:00<00:00, 7.36MB/s]
products_train.csv: 100% 589M/589M [01:13<00:00, 7.96MB/s]
sessions_train.csv: 100% 259M/259M [00:40<00:00, 6.39MB/s]


In [ ]:
import os
import numpy as np
import pandas as pd
from functools import lru_cache

In [ ]:
train_data_dir = '.'
test_data_dir = '.'
task = 'task2'
PREDS_PER_SESSION = 100

In [ ]:
@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(os.path.join(train_data_dir, 'products_train.csv'))

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(os.path.join(train_data_dir, 'sessions_train.csv'))

@lru_cache(maxsize=3)
def read_test_data(task):
    return pd.read_csv(os.path.join(test_data_dir, f'sessions_test_{task}.csv'))

In [ ]:
def read_locale_data(locale, task):
    products = read_product_data().query(f'locale == "{locale}"')
    sess_train = read_train_data().query(f'locale == "{locale}"')
    sess_test = read_test_data(task).query(f'locale == "{locale}"')
    return products, sess_train, sess_test

def show_locale_info(locale, task):
    products, sess_train, sess_test = read_locale_data(locale, task)

    train_l = sess_train['prev_items'].apply(lambda sess: len(sess))
    test_l = sess_test['prev_items'].apply(lambda sess: len(sess))

    print(f"Locale: {locale} \n"
          f"Number of products: {products['id'].nunique()} \n"
          f"Number of train sessions: {len(sess_train)} \n"
          f"Train session lengths - "
          f"Mean: {train_l.mean():.2f} | Median {train_l.median():.2f} | "
           f"Min: {train_l.min():.2f} | Max {train_l.max():.2f} \n"
          f"Number of test sessions: {len(sess_test)}"
        )
    if len(sess_test) > 0:
        print(
             f"Test session lengths - "
            f"Mean: {test_l.mean():.2f} | Median {test_l.median():.2f} | "
            f"Min: {test_l.min():.2f} | Max {test_l.max():.2f} \n"
        )
    print("======================================================================== \n")

In [ ]:
products = read_product_data()
locale_names = products['locale'].unique()
for locale in locale_names:
    show_locale_info(locale, task)

Locale: DE 
Number of products: 518327 
Number of train sessions: 1111416 
Train session lengths - Mean: 57.89 | Median 40.00 | Min: 27.00 | Max 2060.00 
Number of test sessions: 0

Locale: JP 
Number of products: 395009 
Number of train sessions: 979119 
Train session lengths - Mean: 59.61 | Median 40.00 | Min: 27.00 | Max 6257.00 
Number of test sessions: 0

Locale: UK 
Number of products: 500180 
Number of train sessions: 1182181 
Train session lengths - Mean: 54.85 | Median 40.00 | Min: 27.00 | Max 2654.00 
Number of test sessions: 0

Locale: ES 
Number of products: 42503 
Number of train sessions: 89047 
Train session lengths - Mean: 48.82 | Median 40.00 | Min: 27.00 | Max 792.00 
Number of test sessions: 8177
Test session lengths - Mean: 50.23 | Median 40.00 | Min: 27.00 | Max 396.00 


Locale: FR 
Number of products: 44577 
Number of train sessions: 117561 
Train session lengths - Mean: 47.25 | Median 40.00 | Min: 27.00 | Max 687.00 
Number of test sessions: 12521
Test session l

In [ ]:
products.sample(5)

,id,locale,title,price,brand,color,size,model,material,author,desc
1217851,B08VCPPCC2,UK,Fifty Shades 3-Movie Bundle,40000000.07,NaN,NaN,NaN,NaN,NaN,Various,NaN
783283,B07ZNSF7BD,JP,BUFFALO USB3.2(Gen.1)タイオウソトヅケHDD 2TB ブラック,12799.00,バッファロー,ブラック,2TB,HD-LE2U3-BA,NaN,NaN,USB3.2(Gen.1)対応外付けHDD 2TB ブラック
1076680,B09WNJ53G7,UK,Beats Studio Buds – True Wireless Noise Cancel...,159.00,Beats by Dr. Dre,Moon Grey,one size,MMT93ZM/A,Plastic,NaN,Beats wireless headphones and earphones are co...
675960,B096DHH97Z,JP,"Apple iMac Retina 5K 2020(27インチ,8GB RAM,256GB ...",40000000.07,Apple Computer,NaN,RAM:8GB/ SSD:256GB/ CPU:3.1GHz,NaN,NaN,NaN,オーディオ:ステレオスピーカー / 高い信号対雑音比と指向性ビームフォーミングを持つ、スタジ...
267765,B003P4UADU,DE,"ritter Allesschneider solida 4, elekritscher A...",124.99,Ritter,Silbermetallic,NaN,solida4,Metall,NaN,"Großflächiger, abnehmbarer Leichtlaufschlitten..."


In [ ]:
train_sessions = read_train_data()
train_sessions.sample(5)

,prev_items,next_item,locale
1829386,['B09FT69BQ4' '4046059680' 'B0BF3VZQ5C' 'B088T...,B09NRT7Q81,JP
775212,['B0779VFFY6' 'B07G2641MH' 'B0779VFFY6' 'B08LG...,B0B886DKRL,DE
2450058,['B07BHRC65M' 'B07BHRC14Y' 'B07BHRC65M'],B07BHQXV2K,UK
2160732,['B09G97RDQ3' 'B09G9MT326' 'B09G9CXKBS' 'B09G9...,B00IXIWV1M,UK
2718398,['B07F6DHHB1' 'B09X9954P4' 'B07XJ48TP1'],B09Q24NSNK,UK


In [ ]:
test_sessions = read_test_data(task)
test_sessions.sample(5)

,prev_items,locale
18770,['B0BCD4TG1N' 'B0B31X6NKB' 'B0B31X6NKB'],FR
1080,['B07NQ6MD88' 'B09SZ9S2QQ'],ES
30548,['B08XNHNJ37' 'B084QTNJBP' 'B08TLNDLYH'],IT
1937,['B0937L388X' 'B0937GTWWD' 'B0937LDQ5K'],ES
31509,['B015NCZCT8' 'B085CBP9X5'],IT


In [ ]:
def random_predicitons(locale, sess_test_locale):
    random_state = np.random.RandomState(42)
    products = read_product_data().query(f'locale == "{locale}"')
    predictions = []
    for _ in range(len(sess_test_locale)):
        predictions.append(
            list(products['id'].sample(PREDS_PER_SESSION, replace=True, random_state=random_state))
        )
    sess_test_locale['next_item_prediction'] = predictions
    sess_test_locale.drop('prev_items', inplace=True, axis=1)
    return sess_test_locale

In [ ]:
test_sessions = read_test_data(task)
predictions = []
test_locale_names = test_sessions['locale'].unique()
for locale in test_locale_names:
    sess_test_locale = test_sessions.query(f'locale == "{locale}"').copy()
    predictions.append(
        random_predicitons(locale, sess_test_locale)
    )
predictions = pd.concat(predictions).reset_index(drop=True)
predictions.sample(5)

,locale,next_item_prediction
9098,FR,"[B00CQCMZHG, B097PPJQG8, B093D6RQPM, B09PXYWPD..."
31891,IT,"[B08GDC3534, B096GLNLG5, B09TMF6742, B006KCLNU..."
26518,IT,"[B00XI26RP4, B0020MMCVG, B0BCDC2R3D, B087C9NGT..."
25450,IT,"[B0B219QVZ3, B097XPPWXG, B008MJ0X3U, B08HQTFYD..."
27876,IT,"[B09NKLZYCY, B0B2V7K4YX, B06Y1YBBMV, B09HL2HLM..."


In [ ]:
def check_predictions(predictions, check_products=False):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_sessions['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_sessions.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"

        if check_products:
            # This check is not done on the evaluator
            # but you can run it to verify there is no mixing of products between locales
            # Since the ground truth next item will always belong to the same locale
            # Warning - This can be slow to run
            products = read_product_data().query(f'locale == "{locale}"')
            predicted_products = np.unique( np.array(list(preds_locale["next_item_prediction"].values)) )
            assert np.all( np.isin(predicted_products, products['id']) ), f"Invalid products in {locale} predictions"

In [ ]:
check_predictions(predictions)

In [ ]:
predictions.to_parquet(f'submission_{task}.parquet', engine='pyarrow')

In [ ]:
!aicrowd submission create -c task-2-next-product-recommendation-for-underrepresented-languages -f "submission_task2.parquet"

submission_task2.parquet ━━━━━━━━━━━━ 100.0% • 26.8/26.8 MB • 2.8 MB/s • 0:00:00
                                                                                        ╭─────────────────────────╮                                                                                        
                                                                                        │ Successfully submitted! │                                                                                        
                                                                                        ╰─────────────────────────╯                                                                                        
                                                                                              Important links                                                                                              
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────────